## Hidden Markov Models (HMM)

Arjun Ganesh

HMM are popular approaches to working with time series problems where there are latent or hidden states

There are 4 common inference problems in HMM:

- __Filtering__  Inferring the present - carried out by passing messages up and to the right, so infer $h_t$ from $p(h_t|v_{1:t})$  where  $t = T$

- __Prediction__ Inferring the future - similar to filtering but with a new future state, so infer $h_t$ from $p(h_t|v_{1:T})$ where $t>T$

- __Smoothing__  Inferring the past - combine filtering messages with messages up and to the left, so infer $h_t$ from $p(h_t|v_{1:T})$ where $t <T$

- __Decoding__ Find the most likely hidden path - computed similarly to smoothing, so infer the most likely hidden sequence $h_{1:T}$ from $p(h_{1:T}|v_{1:T})$

In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 12 20:35:27 2020

@author: donaldbrown
"""

import numpy as np
import pandas as pd

class HMM:
    """Creates a class for Hidden Markov Models
    Input:
        Viz:      List of observed or visible states over time
        Trans_M:  Numpy array of the Transition matrix for hidden states, 
                  H X H, H=len(Trans_M), no. of hidden variables
        Obs_M:    Numpy array of the Observation matrix, 
                  H X V, V = no. of visible variables
        Pi:       List of initial state probabilities
    Methods:
        filter = The posterior probabilities for hidden states for each time period, T X H array
        smoother = The probabiliteis for the hidden states at each prevoius time period, T X H array
        viturbi = The most likely path of hidden states given the observed state, data frame, 1 X T
        predictor = The probabilities for next hidden state and the next observed state, 1 X H array """
    
    def __init__(self,Viz, Trans_M, Obs_M, Pi):
        # initialize variables
        # Hidden state transition matrix
        self.Trans_M = Trans_M
        # Visible or observates state probabilities given the hidden states
        self.Obs_M = Obs_M
        # No. of hidden states
        self.H = Trans_M.shape[0]
        # No. of observed states
        self.V = Obs_M.shape[0]
        # prior probabaiities for the hidden states
        self.Pi = Pi
        # List of observed states over time
        self.Viz = Viz

        
    def filter(self):
        
        T = len(self.Viz)
        
        # Obtain the joint probabilities of the hidden and observed states at time t
        self.alpha = np.zeros((T, self.H))
        self.alpha[0, :] = self.Pi * self.Obs_M[:,self.Viz[0]]
 
        for t in range(1, T):
            for j in range(self.H):
                self.alpha[t, j] = self.alpha[t - 1].dot(self.Trans_M[:, j]) * self.Obs_M[j, self.Viz[t]]
        
        ### Insert your code here to compute the posterior probabilities ###
        
        self.Post = np.zeros((T, self.H)) 
        
        
        for t in range(T):
            for j in range(self.H):
                self.Post[t, j] = self.alpha[t,j] / np.sum(self.alpha,axis=1)[t]
                
    
        print("self.alpha")
        print(self.alpha)
        print("Posterior")
        print(self.Post)   
        return self.Post
      
    def smoother(self):

        T = len(self.Viz)
        beta = np.zeros((T, self.H))
 
        # setting beta(T) = 1
        beta[T - 1] = np.ones((self.H))
 
        # Loop backwards way from T-1 to 1
        # Due to python indexing the actual loop will be T-2 to 0
        for t in range(T- 2, -1, -1):
            for j in range(self.H):
                beta[t, j] = (beta[t + 1] * self.Obs_M[:, self.Viz[t + 1]]).dot(self.Trans_M[j, :])
                
        # Obtain the posterior probabilities of the hidden states given the observed states       
        
        ### Insert your code here to compute the posterior probabilities ###
        
        Post = np.zeros((T, self.H))
        temp = np.sum(self.alpha*beta,axis=1)
        for t in range(T):
            for j in range(self.H):
                Post[t,j] = self.alpha[t,j] * beta[t,j] / temp[t] 
                
        
        print("beta")
        print(beta)
        print("Posterior")
        print(Post)
 
        return Post
    
    
    def viturbi(self):
        T = len(self.Viz)
        
        # Obtain the joint probabity of the most likely path that ends in state j at time t
        delta = np.zeros((T, self.H))
        delta[0, :] = (self.Pi * self.Obs_M[:, Viz[0]])
 

        prev = np.zeros((T, self.H))
        prev[0,:] = np.repeat(None, 3)
 
        for t in range(1, T):
            for j in range(self.H):
                # The most likely state given our previoius state at t-1
                
                prob = delta[t - 1]* (self.Trans_M[:, j])
 
                #  The probability of the most probable state given the previous state and the observation at time t
                
                delta[t, j] = np.max(prob) * (self.Obs_M[j, Viz[t]])                
                
                # The most probable state given previous state 

                prev[t, j] = np.argmax(prob)
 
                
        # Path Array
        S = np.zeros(T)
 
        # Find the most probable last hidden state
        last_state = np.argmax(delta[T-1, :])
 
        S[T-1] = last_state
        
        # Find the most probable hidden states at the previous times
        ### Insert your code here ###
            
        # Change to states numbers in problem (i.e., +1)
        S = S+1
            
        S = S.reshape([1,3])
 
        # Path, S, as a dataframe 
        # Create a list of column names, Time  
        cols = list()
        for i in range(1,T+1):
            cols.append("Time "+(str(i)))
        Path = pd.DataFrame(S, columns = cols)
        print('delta')
        print(delta)
        print('Previous')
        print(prev)        
        print("Path")
        print(Path)
        return Path
 

    def predictor(self, steps = 1):
        T = len(self.Viz)
        # Hidden state prediction probabilities using filtering results (Post)
        Pred_Hidden = np.dot(self.Post, self.Trans_M)[T-1]
        print("Predicted Hidden State")
        print(Pred_Hidden)
        # Visible state prediction using the predicted hidden state probabilities
        Pred_Visible = np.dot(Pred_Hidden, self.Obs_M)
        print("Predicted Visible State")
        print(Pred_Visible)


## HW 5 Problem - Investment Decision

In [16]:
# Example lecture problem: Burglar in an apartment
# Data 
# Transition matrix
TM = np.array([[.6 ,.3, .1],[.4 ,.4 ,.2],[.1 ,.4 ,.5]])  
# Observation matrix
OM = np.array([[.45 ,.4 ,.15],[.3, .4, .3],[.2,.5,.3]])
OM = OM.T
# Prior probabilities of hidden states
p = [1/3,1/3,1/3]
# Observed visible states
Viz = [0,0,2]


In [17]:
# a) Filtering results

hmm1 = HMM(Viz, TM, OM, p)
hmm1.filter()



self.alpha
[[0.15       0.13333333 0.05      ]
 [0.06675    0.04733333 0.01      ]
 [0.01199667 0.02147917 0.0063425 ]]
Posterior
[[0.45       0.4        0.15      ]
 [0.53794493 0.38146407 0.080591  ]
 [0.301285   0.53942907 0.15928592]]


array([[0.45      , 0.4       , 0.15      ],
       [0.53794493, 0.38146407, 0.080591  ],
       [0.301285  , 0.53942907, 0.15928592]])

.53 is most likely hidden state for time 3. 

In [18]:
# b) Smoothing results
hmm1.smoother()


beta
[[0.12735 0.1195  0.09565]
 [0.3     0.34    0.37   ]
 [1.      1.      1.     ]]
Posterior
[[0.47974133 0.40015068 0.12010799]
 [0.50290905 0.40416893 0.09292202]
 [0.301285   0.53942907 0.15928592]]


array([[0.47974133, 0.40015068, 0.12010799],
       [0.50290905, 0.40416893, 0.09292202],
       [0.301285  , 0.53942907, 0.15928592]])

time 1: .47
time 2: .50
time 3: .54

In [19]:
# c) Decoding results using the Viturbi algorithm

hmm1.viturbi()


delta
[[0.15       0.13333333 0.05      ]
 [0.0405     0.02133333 0.004     ]
 [0.00486    0.006075   0.00128   ]]
Previous
[[nan nan nan]
 [ 0.  1.  1.]
 [ 0.  0.  1.]]
Path
   Time 1  Time 2  Time 3
0     1.0     1.0     2.0


,Time 1,Time 2,Time 3
0,1.0,1.0,2.0


in trouble, in trouble, static

In [20]:
# d) One step predictions of hidden and visible states

hmm1.predictor()

Predicted Hidden State
[0.41247122 0.3698715  0.21765728]
Predicted Visible State
[0.36620924 0.33698715 0.33272718]


Most likely hidden state is in trouble of .41. 
All three visible states are equally as likely in the next time period. 